In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader , Dataset
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

## Создание данных

In [2]:
train_data_path = './train/'

In [3]:
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std = [0.229,0.224,0.225])
])


train_data = torchvision.datasets.ImageFolder(root = train_data_path, transform=transform)

In [4]:
val_data_path = './val/'
val_data = torchvision.datasets.ImageFolder(root = val_data_path, transform= transform)

In [5]:
test_data_path = './test/'
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)

In [6]:
batch_size = 64
train_data_loader = DataLoader(train_data, batch_size=batch_size)
val_data_loader = DataLoader(val_data, batch_size= batch_size)
test_data_loader = DataLoader(test_data, batch_size= batch_size)

In [7]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84,50)
        self.fc3 = nn.Linear(50,2)
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x),dim=1)
        return x

In [8]:
simple_net = SimpleNet()

In [9]:
optimizer = optim.Adam(simple_net.parameters(),lr=0.001)
epochs = 10

In [10]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
simple_net.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [11]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=10,device='cpu'):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
            
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                output = model(inputs)
                loss = loss_fn(output,targets) 
                valid_loss += loss.data.item() * inputs.size(0)
                
                correct = torch.eq(torch.max(output, dim=1)[1], targets).view(-1)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
                
            valid_loss /= len(val_loader.dataset)

        print(f'Epoch: {epoch + 1}, Training Loss: {training_loss:.2f}, Validation Loss: {valid_loss:.2f}, Accuracy: {num_correct / num_examples:.2f}')

In [19]:
train(simple_net,optimizer,nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=10, device=device)

Epoch: 1, Training Loss: 0.66, Validation Loss: 0.57, Accuracy: 0.75
Epoch: 2, Training Loss: 0.59, Validation Loss: 0.68, Accuracy: 0.63
Epoch: 3, Training Loss: 0.62, Validation Loss: 0.61, Accuracy: 0.70
Epoch: 4, Training Loss: 0.53, Validation Loss: 0.63, Accuracy: 0.67
Epoch: 5, Training Loss: 0.54, Validation Loss: 0.66, Accuracy: 0.64
Epoch: 6, Training Loss: 0.50, Validation Loss: 0.68, Accuracy: 0.63
Epoch: 7, Training Loss: 0.53, Validation Loss: 0.64, Accuracy: 0.66
Epoch: 8, Training Loss: 0.49, Validation Loss: 0.61, Accuracy: 0.70
Epoch: 9, Training Loss: 0.56, Validation Loss: 0.56, Accuracy: 0.73
Epoch: 10, Training Loss: 0.46, Validation Loss: 0.65, Accuracy: 0.66


In [ ]:
labels = ['cat','fish']

img = Image.open("./test/fish/Ace_Charters.jpg") 

img = transform(img)  # (C, H, W)
img = img.unsqueeze(0).to(device) 

simple_net.eval()

with torch.no_grad():
    prediction = simple_net(img)  # (1, num_classes)
    prediction = F.softmax(prediction, dim=1)
    prediction = prediction.argmax(dim=1).item() 

print(labels[prediction])

fish
